# Land cover classification at the Mississppi Delta

In this notebook, you will use a k-means **unsupervised** clustering
algorithm to group pixels by similar spectral signatures. **k-means** is
an **exploratory** method for finding patterns in data. Because it is
unsupervised, you don’t need any training data for the model. You also
can’t measure how well it “performs” because the clusters will not
correspond to any particular land cover class. However, we expect at
least some of the clusters to be identifiable as different types of land
cover.

You will use the [harmonized Sentinel/Landsat multispectral
dataset](https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf).
You can access the data with an [Earthdata
account](https://www.earthdata.nasa.gov/learn/get-started) and the
[`earthaccess` library from
NSIDC](https://github.com/nsidc/earthaccess):

## STEP 1: Set up

### Step 1a: Load libraries and set GDAL parameters

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Import all libraries you will need for this analysis</li>
<li>Configure GDAL parameters to help avoid connection errors:
<code>python      os.environ["GDAL_HTTP_MAX_RETRY"] = "5"      os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"</code></li>
</ol></div></div>

In [ ]:
# Use "pip install ____" in the terminal to install missing libraries
# Partition the libraries into smaller cells to avoid longer wait times

# Import the below libraries:
# For working with disk data 
import os # To create reproducible file paths
import pickle # To serialize and reserialize objects (save and load objects from disk)

# For visual displays within VS Code
import warnings # To display warnings

# For the progress bar
from IPython.display import display
from ipywidgets import IntProgress
from tqdm.notebook import tqdm

# For lazy loading
import dask
import gc

# For API access
import earthaccess # To access satellite imagery through the NASA API

# To work with different types of data and conduct data analysis
import cartopy.crs as ccrs # To project coordinates for spatial data and mapping 
import earthpy as et # For spatial data analysis
import geopandas as gpd # To handle vectors/shapefiles
import geoviews as gv # To process visualizations
import numpy as np # To work with arrays
import pandas as pd # To work with tables
import re # For regular expressions; regular expression matching operations
import rioxarray as rxr # To work with raster data
import rioxarray.merge as rxrmerge # To merge rasters
from shapely.geometry import Polygon # To work with polygons
from sklearn.cluster import KMeans # For k-means clustering
import xarray as xr # To work with data arrays

# For hvPlot visualization
import hvplot.pandas # To generate plots from Pandas dataframes
import hvplot.xarray # To enable visualization of xarray objects

# Set the GDAL parameters to avoid interruptions
os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"

# Hide non-critical warnings
warnings.simplefilter('ignore')

### Step 1b: Run the caching decorator

Below you can find code for a caching **decorator** which you can use in
your code. To use the decorator:

``` python
@cached(key, override)
def do_something(*args, **kwargs):
    ...
    return item_to_cache
```

This decorator will **pickle** the results of running the
`do_something()` function, and only run the code if the results don’t
already exist. To override the caching, for example temporarily after
making changes to your code, set `override=True`. Note that to use the
caching decorator, you must write your own function to perform each
task!

You might notice that typically in these assignments, we start by creating a data_dir to store our data files. Here, our caching decorator is making the data directory for us.

In [ ]:
# A decorator modifies or extends another function
# Pickling the function results ensures the function only runs if the output does not exist
 
# Make the caching decorator:
# Define the decorator
# Set up the rules for the caching; base name for caching = "func_key"
# Set the override to false to avoid reruns
def cached(func_key, override=False):
    # Dox strings explain what the function does
    # Establish the parameters for the function
    """
    A decorator to cache function results
    
    Parameters
    ==========
    key: str
      File basename used to save pickled results
    override: bool
      When True, re-compute even if the results are already stored
    """
    # Create the function that computes and saves or loads the results
    def compute_and_cache_decorator(compute_function):
        """
        Wrap the caching function
        
        Parameters
        ==========
        compute_function: function
          The function to run and cache results
        """
        # Create a function that accepts positional arguments
        # args = Arguments that are defined by their name in the function call
        def compute_and_cache(*args, **kwargs):
            """
            Perform a computation and cache, or load cached result.
            
            Parameters
            ==========
            args
              Positional arguments for the compute function
            kwargs
              Keyword arguments for the compute function
            """
            # Add an identifier ("cache_key") from the particular function call
            # If pass, build a single string by joining func_key with kwargs
            # If no cache_key value is given, provide base name only
            if 'cache_key' in kwargs:
                key = '_'.join((func_key, kwargs['cache_key']))
            else:
                key = func_key

            # Define a file path based on the directory structure in earthpy
            path = os.path.join(
                
                # Establish the earthpy directory
                et.io.HOME, 
                
                # Establish the earthpy dataset
                et.io.DATA_NAME, 
                
                # Make a subdirectory called "jars"
                'jars', 
                
                # Use f-string (formatted string) to create a string by embedding the value of the variable "key" into the string 
                # Use .pickle file extension (
                # Pickle file is a serialized python object
                f'{key}.pickle')
            
            # Check if the cache exists already or if caching should be overriden
            if not os.path.exists(path) or override:
                
                # Make a jars directory if needed
                os.makedirs(os.path.dirname(path), exist_ok=True)
                
                # Run the compute function as the user did
                result = compute_function(*args, **kwargs)
                
                # Pickle the object (save to file)
                # Open the cache file at file name
                with open(path, 'wb') as file:
                    
                    # Save the result without needing to recompute when loading it back into Python
                    pickle.dump(result, file)
            
            ### If the file already exists/the cache is not being overriden:
            else:
               
                # Unpickle the object (load the cached result)
                with open(path, 'rb') as file:
                    
                    # Use pickle.load to unserialize the file back into a Python object
                    result = pickle.load(file)

            # Return either the computed result or cached result    
            return result
        
        # Return wrapper function
        return compute_and_cache
    
    # Return decorator configured by func_key and override
    return compute_and_cache_decorator

## STEP 2: Study site

For this analysis, you will use a watershed from the [Water Boundary
Dataset](https://www.usgs.gov/national-hydrography/access-national-hydrography-products),
HU12 watersheds (WBDHU12.shp).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Download the Water Boundary Dataset for region 8 (Mississippi)</li>
<li>Select watershed 080902030506</li>
<li>Generate a site map of the watershed</li>
</ol>
<p>Try to use the <strong>caching decorator</strong></p></div></div>

We chose this watershed because it covers parts of New Orleans an is
near the Mississippi Delta. Deltas are boundary areas between the land
and the ocean, and as a result tend to contain a rich variety of
different land cover and land use types.

In [ ]:
# A level 12 watershed is the most detailed, granular way of dividing the watershed dataset
# Assign the hydrologic unit code
HUC_LEVEL = 12

# Using the cache decorator, download, unzip, and read the shapefile:
# Use @ to call the decorator
@cached(f'wbd_08_hu{HUC_LEVEL}_gdf')

# Make a function to read the file
def read_wbd_file(wbd_filename, cache_key):
    # Define the URL data is being pulled from
    wbd_url = (
        "https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/WBD/HU2/Shape/"
        # Insert the name of the desired file
        f"{wbd_filename}.zip"
    )

    # Download the data and unzip it into the directory
    wbd_dir = et.data.get_data(url = wbd_url)

    # Create a path to the shapefile in the directory
    wbd_path = os.path.join(wbd_dir,
                            'Shape',                            
                            f'WBDHU{HUC_LEVEL}.shp')
    
    # Read the shapefile as a GeoDataFrame (GDF)
    wbd_gdf = gpd.read_file(wbd_path,    
                            # Use the pyogrio library
                            engine = 'pyogrio')
    
    # Return the GDF for the watershed boundaries
    return wbd_gdf

In [ ]:
# Open the shapefile using the created wbd_read_file
wbd_gdf = read_wbd_file("WBD_08_HU2_SHAPE",
                            f'hu{HUC_LEVEL}')

# Open the shapefile to see the results
wbd_gdf

,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,states,...,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID,geometry
0,{8AFB1AF9-7296-4303-89DE-14CD073B859A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"535297,540579",29441.81,119.15,LA,...,Gourd Bayou-Youngs Bayou,S,"LE,ID,DD",080500011308,0.0,0.0,NaN,NaN,1,"POLYGON ((-92.00021 32.53586, -91.99994 32.535..."
1,{916A17A6-B4A0-4FD7-9BB8-FFD1936B15B2},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,535512,11406.67,46.16,LA,...,Hams Creek,S,ID,080802050104,0.0,0.0,NaN,NaN,2,"POLYGON ((-93.37574 30.58982, -93.3747 30.5891..."
2,{493C7EC1-2F1C-4B84-AFFB-6F6868A9868E},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547190,559640",29138.21,117.92,LA,...,Caney Creek-Bayou D'Arbonne,S,NM,080402060503,0.0,0.0,NaN,NaN,3,"POLYGON ((-93.07761 32.88752, -93.07784 32.887..."
3,{49A3C087-B460-4F97-9D99-78CBB675248B},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77417,78285",17759.39,71.87,AR,...,L'Aigle Creek-Saline River,S,NM,080402020206,0.0,0.0,NaN,NaN,4,"POLYGON ((-92.08947 33.29383, -92.0897 33.2938..."
4,{0FB41498-11EA-4AB1-AF05-E2A8E5E2E274},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,1628466,98564.62,398.88,LA,...,West Cote Blanche Bay,W,NM,080801030800,0.0,0.0,NaN,NaN,5,"POLYGON ((-91.62408 29.73947, -91.62195 29.737..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,{9E524E78-2605-48CB-A41F-618AFCDF513D},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"695171,692611",9132.46,36.96,MS,...,Widow Bayou-Big Sunflower River,S,"AW,LE,TF",080302071707,0.0,0.0,NaN,NaN,2637,"POLYGON ((-90.76273 32.97428, -90.76209 32.973..."
2637,{C11913D9-C534-4755-884C-4CAD470ED143},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"77280,81842",20587.50,83.31,AR,...,Lindermans Lake-Bayou Des Arc,S,LE,080203010306,0.0,0.0,NaN,NaN,2638,"POLYGON ((-91.73427 34.99197, -91.7342 34.9923..."
2638,{3EEBF422-01AC-4322-A63C-24C0A34E1E4F},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,714675,21405.46,86.62,MO,...,Brewer Lake,S,"LE,DD,IT,TF",080103000102,0.0,0.0,NaN,NaN,2639,"POLYGON ((-89.13715 36.97285, -89.13387 36.970..."
2639,{4734715C-0F4A-4211-BBAE-86605B20B79A},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"547104,553857",38848.09,157.21,LA,...,Blounts Creek-Calcasieu River,S,ID,080802030302,0.0,0.0,NaN,NaN,2640,"POLYGON ((-92.75965 31.12593, -92.75881 31.125..."


In [ ]:
# Filter/restrict the shapefile to the watershed being used
# Define the GDF for the watershed by subsetting the GDF for the entire dataset
delta_gdf = wbd_gdf[wbd_gdf[
                            # Filter the GDF to the rows of the selected watershed
                            # Hydrologic unit code (HUC) for portion of Mississippi is 080902030506
                            # Dissolve to exclude individual rows
                            f'huc{HUC_LEVEL}'].isin(['080902030506'])].dissolve()

# Display the data
delta_gdf

,geometry,tnmid,metasource,sourcedata,sourceorig,sourcefeat,loaddate,referenceg,areaacres,areasqkm,...,huc12,name,hutype,humod,tohuc,noncontrib,noncontr_1,shape_Leng,shape_Area,ObjectID
0,"POLYGON ((-89.97047 29.74687, -89.96593 29.750...",{E942B72E-599E-48F5-908A-EA5265701C14},{511D2AC8-11BA-45FC-AB98-F69D693D4C44},Watershed Boundary Dataset (WBD),Natural Resources and Conservation Service and...,None,2024-08-15,"536881,539539",37355.86,151.17,...,080902030506,Manuel Canal-Spanish Lake,D,GC,080902030508,0.0,0.0,NaN,NaN,2560


In [ ]:
# Create a site map with satellite imagery in the background 
(
    # Project the delta's GDF to Mercator
    delta_gdf.to_crs(ccrs.Mercator())

    # Use hvPlot
    .hvplot(
        # Make the watershed transparent
        alpha = 0.35, fill_color = "white",
        # Add the satellite basemap
        tiles = "EsriImagery",
        # Plot in Mercator
        crs = ccrs.Mercator())
        
    # Set the plot size
    .opts(width = 600, height = 300)
)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-response"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div></div><div class="callout-body-container callout-body"><p>Write a 2-3 sentence <strong>site description</strong> (with
citations) of this area that helps to put your analysis in context.</p></div></div>


**YOUR SITE DESCRIPTION HERE**

The above hvPlot shows a hydrological feature located to the southeast of New Orleans: the Mississippi River Delta (MRD). Because parts of New Orleans are below sea level, a levee system is used to protect the city from flooding. These measures have contributed to the loss of over 25% of the MRD's coastal wetlands during the 20th century, and climate changed-induced threats like rising sea levels continue to threaten the MRD's ecosystems (Day et al., 2024).

References

Day, J. W., Y. Jun Xu, Keim, B. D., Brown, V. M., Giosan, L., Mann, M. E., & Stephens, J. R. (2024). Emerging climate threats to the Mississippi River Delta: Moving from restoration to adaptation. *One Earth*, *7*(4). https://doi.org/10.1016/j.oneear.2024.03.001

## STEP 3: Multispectral data

### Step 3a: Search for data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Log in to the <code>earthaccess</code> service using your Earthdata
credentials:
<code>python      earthaccess.login(persist=True)</code></li>
<li>Modify the following sample code to search for granules of the
HLSL30 product overlapping the watershed boundary from May to October
2023 (there should be 76 granules):
<code>python      results = earthaccess.search_data(          short_name="...",          cloud_hosted=True,          bounding_box=tuple(gdf.total_bounds),          temporal=("...", "..."),      )</code></li>
</ol></div></div>

In [ ]:
# Log into EarthAccess
earthaccess.login(persist = True)

In [ ]:
# Search for wanted HLS granules 
results = earthaccess.search_data(
    # Specify the desired dataset and spatial resolution
    short_name = "HLSL30",
    # Specify that cloud data is being used
    cloud_hosted = True,
    # Use the bounding box to establish the watershed boundary
    bounding_box = tuple(delta_gdf.total_bounds),
    # Set the temporal range for the data
    temporal = ("2024-06", "2024-08")
)

# View the results
results

[Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -89.79864173, 'Latitude': 29.70347853}, {'Longitude': -89.76643746, 'Latitude': 30.69278312}, {'Longitude': -90.91181412, 'Latitude': 30.71627038}, {'Longitude': -90.93262544, 'Latitude': 29.72659663}, {'Longitude': -89.79864173, 'Latitude': 29.70347853}]}}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-06-07T16:31:11.509Z', 'EndingDateTime': '2024-06-07T16:31:11.509Z'}}
 Size(MB): 169.50417041778564
 Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B10.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.SAA.tif', 'https://data.l

### Step 3b: Compile information about each granule

I recommend building a GeoDataFrame, as this will allow you to plot the
granules you are downloading and make sure they line up with your
shapefile. You could also use a DataFrame, dictionary, or a custom
object to store this information.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each search result:
<ol type="1">
<li>Get the following information (HINT: look at the [‘umm’] values for
each search result):
<ul>
<li>granule id (UR)</li>
<li>datetime</li>
<li>geometry (HINT: check out the shapely.geometry.Polygon class to
convert points to a Polygon)</li>
</ul></li>
<li>Open the granule files. I recommend opening one granule at a time,
e.g. with (<code>earthaccess.open([result]</code>).</li>
<li>For each file (band), get the following information:
<ul>
<li>file handler returned from <code>earthaccess.open()</code></li>
<li>tile id</li>
<li>band number</li>
</ul></li>
</ol></li>
<li>Compile all the information you collected into a GeoDataFrame</li>
</ol></div></div>

In [ ]:
# Create a function to process all the granules and extract information for each granule from the Earth Access search:
# Define the function
def get_earthaccess_links(results):
    # Make and display a progress bar
    f = IntProgress(min = 0, max = len(results), description = "Open granules")
    display(f) # To display the progress bar

    # Use a regular expression to extract tile_id and bank from THE .tif files
    re_url = re.compile(
        r'\.(?P<tile_id>T\w+)\.\d+T\d+\.v\d+\.\d+\.(?P<band>[A-Za-z0-9]+)\.tif$'
    )

    # Accumulate GDF rows from each granule
    link_rows = []

    # Loop over the granules to extract info
    for granule in results:
        # Locate the metadata (Universal metadata model = UMM)
        info_dict = granule['umm']
        # Pull out the unique identifier for the granule
        granule_id = info_dict['GranuleUR']
        # Extract the date and time 
        datetime = pd.to_datetime(
            info_dict['TemporalExtent']['RangeDateTime']['BeginningDateTime']
        )
        # Extact the boundary coordinates for the granule
        points = (
            info_dict
            ['SpatialExtent']['HorizontalSpatialDomain']['Geometry']['GPolygons'][0]['Boundary']['Points']
        )
        # Create a polygon using coordinate points for the granules
        geometry = Polygon(
            [(point['Longitude'],
            point['Latitude']) for point in points]
        )

        # Get the URL and open granule
        files = earthaccess.open([granule])

        # Loop through each file in the granule
        for file in files:
            # Use a URL regular expression to get the URL
            match = re_url.search(file.full_name) 
            # If a match is found, append data to initialized link_rows GDF 
            if match is not None:
                link_rows.append(
                    # Make a GDF with the granule's data and geometry
                    gpd.GeoDataFrame(
                        dict(
                            # For the timestamp
                            datetime = [datetime],
                            # For a unique ID
                            tile_id = [match.group('tile_id')],
                            # For the band name
                            band = [match.group('band')],
                            # For the URL
                            url = [file], 
                            # For the polygon
                            geometry = [geometry],
                            # For the coordinate reference system (CRS)
                            crs = "ESPG:4326"
                        )
                    )
                )

        # Update the progress bar after each granule is done
        f.value+=1

    # Combine into a single GDF
    file_df = pd.concat(link_rows).reset_index(drop = True)   

    # Return the final GDF file
    return file_df

In [ ]:
# View the granule results
granule = results[0]
granule

Collection: {'EntryTitle': 'HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -89.79864173, 'Latitude': 29.70347853}, {'Longitude': -89.76643746, 'Latitude': 30.69278312}, {'Longitude': -90.91181412, 'Latitude': 30.71627038}, {'Longitude': -90.93262544, 'Latitude': 29.72659663}, {'Longitude': -89.79864173, 'Latitude': 29.70347853}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2024-06-07T16:31:11.509Z', 'EndingDateTime': '2024-06-07T16:31:11.509Z'}}
Size(MB): 169.50417041778564
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B10.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.SAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.VZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B06.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B09.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B04.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B03.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B07.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.VAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.Fmask.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B01.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.SZA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B11.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B05.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T15RYP.2024159T163111.v2.0/HLS.L30.T15RYP.2024159T163111.v2.0.B02.tif']

In [ ]:
info_dict = granule ['umm']
info_dict

{'TemporalExtent': {'RangeDateTime': {'BeginningDateTime': '2024-06-07T16:31:11.509Z',
   'EndingDateTime': '2024-06-07T16:31:11.509Z'}},
 'GranuleUR': 'HLS.L30.T15RYP.2024159T163111.v2.0',
 'AdditionalAttributes': [{'Name': 'LANDSAT_PRODUCT_ID',
   'Values': ['LC08_L1TP_022039_20240607_20240607_02_RT']},
  {'Name': 'CLOUD_COVERAGE', 'Values': ['10']},
  {'Name': 'MGRS_TILE_ID', 'Values': ['15RYP']},
  {'Name': 'SPATIAL_COVERAGE', 'Values': ['100']},
  {'Name': 'SPATIAL_RESOLUTION', 'Values': ['30.0']},
  {'Name': 'SPATIAL_RESAMPLING_ALG', 'Values': ['Cubic Convolution']},
  {'Name': 'HLS_PROCESSING_TIME', 'Values': ['2024-06-21T18:26:46Z']},
  {'Name': 'SENSING_TIME', 'Values': ['2024-06-07T16:31:11.5093050Z']},
  {'Name': 'HORIZONTAL_CS_NAME', 'Values': ['UTM, WGS84, UTM ZONE 15']},
  {'Name': 'ULX', 'Values': ['699960.0']},
  {'Name': 'ULY', 'Values': ['3400020.0']},
  {'Name': 'ADD_OFFSET', 'Values': ['0']},
  {'Name': 'REF_SCALE_FACTOR', 'Values': ['0.0001']},
  {'Name': 'THERM_SC

In [ ]:
# Run the function to get the granule search results
""" The library ipywidgets needs to be installed to display the IntProgress bar. This can be done by using the
command "pip install ipywidgets" in the terminal. "from IPython.display import display" and "from ipywidgets 
import IntProgress" are also used in the first cell to help Python process this display. """
file_df = get_earthaccess_links(results)

IntProgress(value=0, description='Open granules', max=44)

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

...

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

### Step 3c: Open, crop, and mask data

This will be the most resource-intensive step. I recommend caching your
results using the `cached` decorator or by writing your own caching
code. I also recommend testing this step with one or two dates before
running the full computation.

This code should include at least one **function** including a
numpy-style docstring. A good place to start would be a function for
opening a single masked raster, applying the appropriate scale
parameter, and cropping.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>For each granule:
<ol type="1">
<li><p>Open the Fmask band, crop, and compute a quality mask for the
granule. You can use the following code as a starting point, making sure
that <code>mask_bits</code> contains the quality bits you want to
consider: ```python # Expand into a new dimension of binary bits bits =
( np.unpackbits(da.astype(np.uint8), bitorder=‘little’)
.reshape(da.shape + (-1,)) )</p>
<p># Select the required bits and check if any are flagged mask =
np.prod(bits[…, mask_bits]==0, axis=-1) ```</p></li>
<li><p>For each band that starts with ‘B’:</p>
<ol type="1">
<li>Open the band, crop, and apply the scale factor</li>
<li>Name the DataArray after the band using the <code>.name</code>
attribute</li>
<li>Apply the cloud mask using the <code>.where()</code> method</li>
<li>Store the DataArray in your data structure (e.g. adding a
GeoDataFrame column with the DataArray in it. Note that you will need to
remove the rows for unused bands)</li>
</ol></li>
</ol></li>
</ol></div></div>

In [ ]:
# Apply the cached decorator to the function
@cached('delta_reflecance_da_df')

# Write a function that computes reflectance data using the search results (dataframe of URLs) and watershed boundary
def compute_reflectance_da(search_results, boundary_gdf):
    """
    Use VSI to connect to the files, crop them, apply a cloud mask, and wrangle the data
    Return a single reflectance dataframe with bands as columns and the centroid coordinates and datetime as the
    index

    Parameters
    search_results: list # The search result links to the files (URLs)
    boundary_gdf: gpd.GeoDataFrame # The boundary is used to crop the data
    """
    # Write a function to open the raster from the URL, apply scale factor, and crop and mask data
    def open_dataarray(url, boundary_proj_gdf, scale = 1, masked = True):
        # Open the raster data
        da = rxr.open_rasterio(url, masked = masked, 
                               # Add chunking
                               chunks={"x": 1024, "y": 1024}
                               ).squeeze() * scale
        # If needed, reproject the boundary to match the raster's CRS
        if boundary_proj_gdf is None:
            boundary_proj_gdf = boundary_gdf.to_crs(da.rio.crs)
        # Crop the raster to the bounding box
        cropped = da.rio.clip_box(*boundary_proj_gdf.total_bounds)
        # Return the cropped raster
        return cropped

    # Write a function to apply a cloud mask
    # Mask out law quality data by bit
    def compute_quality_masked(da, mask_bits = [1,2,3]):
        # Unpack the bits to a new axis
        bits = (
            # Unpack each number into individual bits
            np.unpackbits(
                # Convert to 8-bit unsigned integer
                da.astype(np.uint8),
                # Set the order of the bits
                bitorder = 'little'
            # Reshape to match the original data with an extra dimension for the bits
            ).reshape(da.shape + (-1,))
        )
        # Grab the bits and check if they are flagged
        mask = np.prod(
            bits[
                ..., mask_bits] == 0, 
                     axis = -1
        )
        # Return the mask
        return mask

    # Grab the metadata
    file_df = get_earthaccess_links(search_results)

    # Store results for each granule in a list
    granule_da_rows = []

    # Store the projected boundary
    boundary_proj_gdf = None

    # Group the data for each granule
    group_iter = file_df.groupby(
        # By datetime and tile_id
        ['datetime', 'tile_id']
    )

    # Loop through each image and its metadata
    for (datetime, tile_id), granule_df in tqdm(group_iter):
        # Print the status bar
        print(f'Processing granule {tile_id} {datetime}')
        # Find each granule's cloud mask (fmask) URL
        cloud_mask_url = (
            granule_df.loc[granule_df.band == 'Fmask', 'url']
            .values[0])
        # Open the granule cloud mask
        cloud_masked_cropped_da = open_dataarray(cloud_mask_url, boundary_proj_gdf, masked = False)
        # Compute the cloud mask
        cloud_mask = compute_quality_masked(cloud_masked_cropped_da)
        # Free memory early
        del cloud_masked_cropped_da        
        gc.collect()
        # Loop through each spectral band to open, crop, and mask the band
        da_list = []
        df_list = []

        # Loop through each band in the granule
        for i, row in granule_df.iterrows():
            # Only loop through the spectral bands
            if row.band.startswith(('B')):
                # Open the band's raster and scale to reflectance
                band_cropped = open_dataarray(
                    row.url, 
                    boundary_proj_gdf, 
                    scale = 0.0001
                )
                # Name the raster by the band
                band_cropped.name = row.band
                # Cut raster memory in half
                band_cropped = band_cropped.astype("float16")
                # Apply the cloud mask to the raster
                row['da'] = band_cropped.where(cloud_mask)
                # Append the row to granule_da_rows
                granule_da_rows.append(row.to_frame().T)
                
    # Reassemble the metadata dataframe
    return pd.concat(granule_da_rows)

In [ ]:
# Apply the function
reflectance_da_df = compute_reflectance_da(results, delta_gdf)

IntProgress(value=0, description='Open granules', max=44)

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

...

PROCESSING TASKS | :   0%|          | 0/15 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

Processing granule T15RYN 2024-06-07 16:31:11.509000+00:00


NameError: name 'gc' is not defined

In [ ]:
# Display the dataframe
reflectance_da_df

### Step 3d: Merge and Composite Data

You will notice for this watershed that:   
1. The raster data for each date are spread across 4 granules  
2. Any given image is incomplete because of clouds

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">

*   1. For each band:  
    *   a. For each date:  
        *   i. Merge all 4 granules  
        *   ii. Mask any negative values created by interpolating from the nodata value of -9999 (`rioxarray`) should account for this, but doesn't appear to when merging. If you leave these values in, they will create problems later on
    *   b. Concatenate the merged DataArrays along a new date dimension  
    *   c. Take the mean in the date dimension to create a composite image that fills cloud gaps  
    *   d. Add the band as a dimensions, and give the DataArray a name  
*   2. Concatenate along the band dimension


In [ ]:
# Apply the cache decorator
@cached('delta_reflectance_da')

# Create a function to merge and composite reflectance data from multiple granules
"""The end result will be a single composite reflectance image for each spectral band."""
def merge_and_composite_arrays(granule_da_df):
    # Initialize a list to store composites after processing
    da_list = []

    # Loop over each spectral band
    for band, band_df in granule_da_df.groupby('band'):
        # Create a list for storing merged data arrays (one per date)
        merged_das = []
        # Loop over the dates and times of the image acquisition and merge the granules for each data
        for datetime, date_df in band_df.groupby('datetime'):
            # Merge granules for each date
            merged_da = rxrmerge.merge_arrays(list(date_df))
            # Mask negative values (could be no data or invalid data)
            merged_da = merged_da.where(merged_da > 0)
            # Append to the initialized merged_das list
            merged_das.append(merged_da)
        # Composite images across dates
        composite_da = xr.concat(merged_das,
                                 # Make a datetime dimension
                                 # Calculate the median value across datetimes for the pixels
                                 dim = 'datetime').median('datetime')
        # Assign band number to the attribute of the composite data array
        composite_da['band'] = int(band[1:])
        # Name the composite data array
        composte_da.name = 'reflectance'
        # Add the processed and composite data array to the list
        da_list.append(composite_da)
    # Concatenate the composite data arrays for each band along the band dimension
    return xr.concat(da_list, dim = 'band')

In [ ]:
# Call the function to get the final composite reflectance data 
reflectance_da = merge_and_composite_arrays(reflectance_da_df)

## STEP 4: K-means clustering

Cluster your data by spectral signature using the k-means algorithm.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Convert your DataArray into a <strong>tidy</strong> DataFrame of
reflectance values (hint: check out the <code>.to_dataframe()</code> and
<code>.unstack()</code> methods)</li>
<li>Filter out all rows with no data (all 0s or any N/A values)</li>
<li>Fit a k-means model. You can experiment with the number of groups to
find what works best.</li>
</ol></div></div>

In [ ]:
# Convert the spectral DataArray to a tidy DataFrame
model_df = (reflectance_da,
            # Flatten the array into a long dataframe
            .to_dataframe()
            # Select the reflectance column
            .reflectance
            # Make the column wide; each row will be a pixel location
            # Each column is a spectral band with the reflectance value
            .unstack('band')
)

# Filter out rows with no data
model_df = model_df.drop(columns = [10, 11]).dropna()

# Display the dataframe
model_df

In [ ]:
# Assign the minimum and maximum values
min_values = model_df.min()
max_values = model_df.max()

# Print the minimum and maximum values
print(min_values)
print(max_values)

Now we're reading to fit the k-means clustering model. We can run the fit and prediction functions at the same time because we don't have target data.

In [ ]:
# Initialize the k-means model 
k_means = KMeans(n_clusters = 5)

# Fit the model and predict
prediction = k_means.fit_predict(model_df.values)

# Add the predicted values back to the model dataframe
model_df['clusters'] = prediction

# Display the dataframe
model_df

## STEP 5: Plot

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Create a plot that shows the k-means clusters next to an RGB image of
the area. You may need to brighten your RGB image by multiplying it by
10. The code for reshaping and plotting the clusters is provided for you
below, but you will have to create the RGB plot yourself!</p>
<p>So, what is <code>.sortby(['x', 'y'])</code> doing for us? Try the
code without it and find out.</p></div></div>

In [ ]:
# Make data array with bands to use for red, green, and blue (rgb)
rgb = reflectance_da.sel(band = [4, 3, 2])

# Plot rgb
(
    rgb_plot.hvplot.rgb(y = 'y',
                    x = 'x',
                    bands = 'band',
                    xaxis = None,
                    yaxis = None)

                    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        cmap="Colorblind", aspect='equal') 
)

In [ ]:
# Stretch the values
rgb_uint8 = (rgb * 255).astype(np.uint8).where(rgb != np.nan)

rbg_uint8

In [ ]:
# Plot rgb from 0-255
(
    rgb_uint8.hvplot.rgb(y = 'y',
                    x = 'x',
                    bands = 'band',
                    xaxis = None,
                    yaxis = None)
)

In [ ]:
# Enhance brightness
rgb_uint8_bright = rgb_uint8 * 10

rbg_uint8_bright

In [ ]:
# Plot the brighter rgb
(
    rgb_uint8_bright.hvplot.rgb(y = 'y',
                    x = 'x',
                    bands = 'band',
                    xaxis = None,
                    yaxis = None)
)

In [ ]:
# Cap the saturation
rgb_sat = rgb_uint8_bright.where(rgb_uint8_bright < 255, 255)

# Replot
(
    rgb_sat.hvplot.rgb(y = 'y',
                    x = 'x',
                    bands = 'band',
                    xaxis = None,
                    yaxis = None)
)

In [ ]:
# Plot the clusters from k-means
(
    model_df.clusters.to_xarray().hvplot(
        x = 'x',
        y = 'y',
        data_aspect = 1,
        xaxis = None,
        yaxis = None
    )
)

In [ ]:
# Deal with sorting
(
    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        x = 'x',
        y = 'y',
        data_aspect = 1,
        xaxis = None,
        yaxis = None
    )
)

In [ ]:
# Plot the rgb plot next to the clustering plot
(
    rgb_sat.hvplot.rgb(y = 'y',
                    x = 'x',
                    bands = 'band',
                    xaxis = None,
                    yaxis = None)
    +
    model_df.clusters.to_xarray().sortby(['x', 'y']).hvplot(
        x = 'x',
        y = 'y',
        data_aspect = 1,
        xaxis = None,
        yaxis = None
    )
)

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Don’t forget to interpret your plot!</p></div></div>

**YOUR PLOT HEADLINE AND DESCRIPTION HERE**